In [1]:
import os, sys
cur_dir = os.getcwd()
basename = os.path.basename(cur_dir)
for _ in range(5):
    if basename != 'OCLog':
        cur_dir = os.path.dirname(cur_dir)
        basename = os.path.basename(cur_dir)
        print(cur_dir, basename)
    else:
        if cur_dir not in sys.path:
            sys.path.append(cur_dir)
            print(sys.path)

C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\openset openset
C:\Users\Bhujay_ROG\MyDev\OCLog\oclog oclog
C:\Users\Bhujay_ROG\MyDev\OCLog OCLog
['C:\\Users\\Bhujay_ROG\\MyDev\\OCLog\\oclog\\openset\\notebooks', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3312.0_x64__qbz5n2kfra8p0\\python39.zip', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3312.0_x64__qbz5n2kfra8p0\\DLLs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_3.9.3312.0_x64__qbz5n2kfra8p0\\lib', 'C:\\Users\\Bhujay_ROG\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0', 'C:\\Users\\Bhujay_ROG\\venv1', '', 'C:\\Users\\Bhujay_ROG\\venv1\\lib\\site-packages', 'C:\\Users\\Bhujay_ROG\\venv1\\lib\\site-packages\\win32', 'C:\\Users\\Bhujay_ROG\\venv1\\lib\\site-packages\\win32\\lib', 'C:\\Users\\Bhujay_ROG\\venv1\\lib\\site-packages\\Pythonwin', 'C:\\Users\\Bhujay_ROG\\MyDev\\OCLog']


In [2]:
import numpy as np
import tensorflow as tf
from tqdm import trange, tqdm, tnrange
from oclog.BGL.bglog import BGLog, get_embedding_layer
from oclog.openset.boundary_loss import euclidean_metric, BoundaryLoss
from oclog.openset.pretraining import LogLineEncoder, LogSeqEncoder, LogClassifier
from oclog.openset.openset import OpenSet

In [3]:
bglog = BGLog(save_padded_num_sequences=False, load_from_pkl=True, )
train_test = bglog.get_tensor_train_test(ablation=1000)
train_data, test_data = train_test

padded_num_seq_df loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgl_padded_num_seq_df.pkl
trained tokenizer, tk, loaded from C:\Users\Bhujay_ROG\MyDev\OCLog\oclog\BGL\data\bgltk.pkl
train_0:, 800
test_0:, 200
train_1:, 800
test_1:, 200
train_2:, 800
test_2:, 200
train_3:, 800
test_3:, 102
4 class does not have 800 records, it has only 628 records
test_4:, 0
5 class does not have 800 records, it has only 165 records
5 class does not have 200 records, it has only 165 records
6 class does not have 800 records, it has only 75 records
6 class does not have 200 records, it has only 75 records
[[1. 0. 0. 0.]
 [1. 0. 0. 0.]]
<BatchDataset element_spec=(TensorSpec(shape=(32, 32, 64), dtype=tf.int32, name=None), TensorSpec(shape=(32, 4), dtype=tf.float32, name=None))>
<BatchDataset element_spec=(TensorSpec(shape=(32, 32, 64), dtype=tf.int32, name=None), TensorSpec(shape=(32, 4), dtype=tf.float32, name=None))>


In [4]:
line_encoder = LogLineEncoder(bglog, chars_in_line=64)
logSeqencer =  LogSeqEncoder(line_in_seq=32)
ptmodel = LogClassifier(line_encoder=line_encoder, seq_encoder=logSeqencer, num_classes=4)
ptmodel.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
hist = ptmodel.fit(train_data, validation_data=test_data, epochs=2) 

vocab_size: 50
Epoch 1/2
100/100 [==============================] - 11s 100ms/step - loss: 0.4636 - accuracy: 0.8128 - precision: 0.8919 - recall: 0.7347 - val_loss: 0.0431 - val_accuracy: 0.9866 - val_precision: 0.9896 - val_recall: 0.9866
Epoch 2/2
100/100 [==============================] - 10s 97ms/step - loss: 0.0713 - accuracy: 0.9787 - precision: 0.9787 - recall: 0.9772 - val_loss: 0.0142 - val_accuracy: 0.9955 - val_precision: 0.9970 - val_recall: 0.9940


In [5]:
oset = OpenSet(4, ptmodel) 
oset.train(train_data, epochs=5)

100/100 [==============================] - 9s 91ms/step - loss: 0.0262 - accuracy: 0.9900 - precision: 0.9909 - recall: 0.9900


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.36it/s]


train_loss: tf.Tensor(7.8816833, shape=(), dtype=float32)
100/100 [==============================] - 9s 91ms/step - loss: 0.0289 - accuracy: 0.9891 - precision: 0.9891 - recall: 0.9891


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.38it/s]


train_loss: tf.Tensor(7.152867, shape=(), dtype=float32)
100/100 [==============================] - 9s 91ms/step - loss: 0.0191 - accuracy: 0.9937 - precision: 0.9937 - recall: 0.9937


...

train_loss: tf.Tensor(7.9611735, shape=(), dtype=float32)
100/100 [==============================] - 9s 91ms/step - loss: 0.0086 - accuracy: 0.9969 - precision: 0.9969 - recall: 0.9969


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.32it/s]


train_loss: tf.Tensor(10.789292, shape=(), dtype=float32)
100/100 [==============================] - 9s 91ms/step - loss: 0.0252 - accuracy: 0.9909 - precision: 0.9912 - recall: 0.9909


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.34it/s]

train_loss: tf.Tensor(14.4153185, shape=(), dtype=float32)
